In [1]:
import torch
import numpy as np

In [2]:
torch.__version__

'1.8.1'

## Some definitions

In [3]:
a = torch.FloatTensor(3,2)

In [4]:
a

tensor([[1.3733e-14, 1.2102e+25],
        [1.6992e-07, 2.9599e+21],
        [2.8175e+20, 1.7566e+25]])

In [5]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [6]:
b = torch.FloatTensor([[1,2,3], [3,2,1]])

In [7]:
b

tensor([[1., 2., 3.],
        [3., 2., 1.]])

In [8]:
b.zero_()

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [9]:
b

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [10]:
c = torch.FloatTensor([[1,2,3], [4,5,6]])


In [11]:
c.sum()

tensor(21.)

## Gradients

In [12]:
v1 = torch.tensor([1.0, 1.0], requires_grad=True)
v2 = torch.tensor([2.0, 2.0])

In [13]:
v_sum = v1 + v2
v_res = (v_sum*2).sum()
v_res

tensor(12., grad_fn=<SumBackward0>)

In [14]:
"""
(v1)
    \
     (+) -- (*2) -- (sum)
    / v_sum          v_res
(v2)

"""

'\n(v1)\n         (+) -- (*2) -- (sum)\n    / v_sum          v_res\n(v2)\n\n'

In [15]:
v1.is_leaf, v2.is_leaf

(True, True)

In [16]:
v_sum.is_leaf, v_res.is_leaf

(False, False)

In [17]:
v1.requires_grad

True

In [18]:
v2.requires_grad

False

In [19]:
v_res.requires_grad

True

### Calculate gradient

In [20]:
v_res.backward()

In [21]:
v1.grad

tensor([2., 2.])

This means that increasing any element of ```v1``` by one, the resulting value of ```v_res``` will grow by two. 

## NN building blocks

In [22]:
import torch.nn as nn

In [23]:
l = nn.Linear(2,5)
v = torch.FloatTensor([1,2])

In [24]:
l(v)

tensor([ 0.6871,  0.6436, -1.6241, -1.1285,  1.3629], grad_fn=<AddBackward0>)

In [25]:
l

Linear(in_features=2, out_features=5, bias=True)

In [26]:
l.state_dict()

OrderedDict([('weight',
              tensor([[-0.5020,  0.5041],
                      [-0.2524,  0.2461],
                      [-0.2024, -0.5022],
                      [-0.0981, -0.2391],
                      [ 0.0685,  0.6957]])),
             ('bias', tensor([ 0.1808,  0.4038, -0.4173, -0.5522, -0.0970]))])

In [27]:
(1 * 0.2615  + 2 * 0.4348 + 0.3353, 
 1 * 0.2027  + 2 * 0.4945 + (-0.4638),
 1 * -0.4220 + 2 * 0.0338 + 0.2583, 
 1 * -0.3141 + 2 * 0.4801 + 0.3405, 
 1 * 0.2592  + 2 * 0.0579 + 0.4810)

(1.4664, 0.7279, -0.09610000000000002, 0.9866000000000001, 0.856)

In [28]:
l(v)

tensor([ 0.6871,  0.6436, -1.6241, -1.1285,  1.3629], grad_fn=<AddBackward0>)

In [29]:
s = nn.Sequential(
    nn.Linear(2, 5), 
    nn.ReLU(), 
    nn.Linear(5, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.Dropout(p=0.3), 
    nn.Softmax(dim=1)
)

In [30]:
s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): Dropout(p=0.3, inplace=False)
  (6): Softmax(dim=1)
)

In [31]:
minibatch = torch.FloatTensor([[1,2], [3,4]])

In [32]:
s(minibatch)

tensor([[0.1018, 0.1018, 0.1018, 0.0817, 0.1167, 0.0546, 0.1018, 0.1237, 0.0976,
         0.1186],
        [0.0672, 0.1010, 0.1010, 0.1010, 0.1158, 0.1010, 0.0707, 0.1157, 0.1010,
         0.1255]], grad_fn=<SoftmaxBackward>)

## Custom layers

In [33]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5,20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax()
        )
        
    def forward(self, x):
        return self.pipe(x)

In [34]:
net = OurModule(num_inputs=2, num_classes = 3)
v = torch.FloatTensor([[2, 3]])

In [35]:
out = net(v)

/Users/raul.reguillo/.pyenv/versions/3.7.4/envs/gym/lib/python3.7/site-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [36]:
print(net)
print(out)

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Softmax(dim=None)
  )
)
tensor([[0.4003, 0.2259, 0.3738]], grad_fn=<SoftmaxBackward>)


```forward()``` is executed on the overriden ```__call__()``` method of the ```nn.Module```. Other functions such as ```__str__()``` and ```__repr__()``` are also overriden in this module. 

In [37]:
import math
from tensorboardX import SummaryWriter

In [38]:
writer = SummaryWriter()
funcs = {"sin": math.sin, 
         "cos": math.cos,
         "tan": math.tan
        }

In [39]:
for angle in range (-360, 360):
    angle_rad = angle * math.pi /180 
    for name, fun in funcs.items():
        val = fun(angle_rad)
        writer.add_scalar(name, val, angle)
writer.close()

In [40]:
!tensorboard --logdir runs --host localhost

TensorFlow installation not found - running with reduced feature set.
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C
